Kenya data submitted to GISADI database as of Novermber 15

In [2]:
import Bio
from Bio.Seq import Seq 
from Bio import SeqIO
import pandas as pd
import plotly.express as px

In [6]:
#merge new datasets

data = pd.read_table("./1673524662328.metadata.tsv")
data1 = pd.read_table("./1673524946413.metadata.tsv")
data2 = pd.read_table("./1673525130747.metadata.tsv")
merged_data = pd.concat([data, data1,data2])

merged_data.to_csv("./kenya_metadata_nextclade2.csv", index=False)

# Submitting Labs

In [9]:
kenya_metadata_nextclade =  pd.read_csv("./data/kenya_metadata_nextclade2.csv",low_memory=False)
data_df = kenya_metadata_nextclade.loc[:,["strain","date","submitting_lab","date_submitted"]]
data_df.rename(columns = {"date":"sampling_date"}, inplace = True)
data_df["sampling_date"] = pd.to_datetime(data_df["sampling_date"],infer_datetime_format=True, format="%d/%m/%Y", dayfirst=True)
data_df["date_submitted"] = pd.to_datetime(data_df["date_submitted"],infer_datetime_format=True, format="%d/%m/%Y", dayfirst=True)

data_df["days_to_submission"] = data_df["date_submitted"] - data_df["sampling_date"]
data_df["days_to_submission"] = data_df["days_to_submission"].apply(lambda x: pd.Timedelta(x).days)

##Filter names to standard
data_df["submitting_lab"] =  data_df["submitting_lab"].replace(["KEMRI-Wellcome Trust Research Programme/KEMRI-CGMR-C Kilifi",
                                "Epidemiology and Demography,  KEMRI-Wellcome Trust Research Programme",
                                ], "KEMRI-Wellcome Trust Research Programme,Kilifi")
data_df["submitting_lab"] = data_df["submitting_lab"].replace('ARI lab, Center for Virus Research, Kenya Medical Research Institute','ARI lab, Center for Virus Research')

data_df["submitting_lab"] =  data_df["submitting_lab"].replace('THE AFRICA GENOMICS CENTRE AND CONSULTANCY','THE AFRICA GENOMICS CENTRE AND CONSULTANCY LIMITED')

data_df["submitting_lab"] =  data_df["submitting_lab"].replace('USAMRD-A, Basic Science Laboratory','United States Army Medical Research Directorate, Kenya')

data_df["submitting_lab"] = data_df["submitting_lab"].replace('CERI, Centre for Epidemic Response and Innovation, Stellenbosch University and KRISP, KZN Research Innovation and Sequencing Platform, UKZN.',"Centre for Epidemic Response and Innovation,UKZN")

data_df["submitting_lab"] =  data_df["submitting_lab"].replace(['CDC-DLSP-Kisumu','Diagnostics and Laboratory Systems Program Lab, KEMRI-CGHR'],'DLSP KEMRI/CDC Lab')

data_df.to_csv(r"../Cemia_Dash/data/submitting_labs.txt", index=None,sep="\t")

In [8]:
data_df.head()

,strain,sampling_date,submitting_lab,date_submitted,days_to_submission
0,hCoV-19/Kenya/C71987/2020,2020-11-17,"KEMRI-Wellcome Trust Research Programme,Kilifi",2021-02-11,86
1,hCoV-19/Kenya/C66133/2020,2020-10-23,"KEMRI-Wellcome Trust Research Programme,Kilifi",2021-02-11,111
2,hCoV-19/Kenya/C80839/2021,2021-01-25,"KEMRI-Wellcome Trust Research Programme,Kilifi",2021-02-11,17
3,hCoV-19/Kenya/C80801/2021,2021-01-15,"KEMRI-Wellcome Trust Research Programme,Kilifi",2021-02-11,27
4,hCoV-19/Kenya/C1751/2020,2020-04-22,"KEMRI-Wellcome Trust Research Programme,Kilifi",2020-06-02,41


In [ ]:
submitting_labs = pd.read_table("./data/submitting_labs.txt")
submitting_labs.head()

,sampling_date,submitting_lab,date_submitted,days_to_submission
0,2020-11-17,"KEMRI-Wellcome Trust Research Programme,Kilifi",2021-02-11,86
1,2020-10-23,"KEMRI-Wellcome Trust Research Programme,Kilifi",2021-02-11,111
2,2021-01-25,"KEMRI-Wellcome Trust Research Programme,Kilifi",2021-02-11,17
3,2021-01-15,"KEMRI-Wellcome Trust Research Programme,Kilifi",2021-02-11,27
4,2020-04-22,"KEMRI-Wellcome Trust Research Programme,Kilifi",2020-06-02,41


## Counting Ns in the Sequences

In [ ]:
metadata = data_df.loc[:,["strain","date_submitted","submitting_lab"]]

fasta_file = "./data/all.kenya.fasta"
 # create a dictionary
ids = []
Ns = []
with open(fasta_file) as file:
    for seq_record in SeqIO.parse(file,"fasta"):
        ids.append(seq_record.id)
        Ns.append(seq_record.seq.count("N"))
count_Ns = dict(zip(ids,Ns))
Kenya_Ns = pd.DataFrame.from_dict(count_Ns,orient="index").reset_index().rename(columns = {"index":"Seq_ID", 0:"N_count"})
Kenya_Ns["n_percentage"] = Kenya_Ns["N_count"].apply(lambda x: x/29903*100)
Kenya_Ns = pd.merge(Kenya_Ns, metadata, left_on = "Seq_ID", right_on = "strain",how = "outer")
Kenya_Ns.drop("strain", axis=1, inplace=True)
Kenya_Ns.to_csv("../Cemia_Dash/data/count_ns.csv")

In [ ]:
Kenya_Ns.head()

,Seq_ID,N_count,n_percentage,date_submitted,submitting_lab
0,hCoV-19/Kenya/C71987/2020,847,2.832492,2021-02-11,"KEMRI-Wellcome Trust Research Programme,Kilifi"
1,hCoV-19/Kenya/C66133/2020,4596,15.369695,2021-02-11,"KEMRI-Wellcome Trust Research Programme,Kilifi"
2,hCoV-19/Kenya/C80839/2021,5222,17.463131,2021-02-11,"KEMRI-Wellcome Trust Research Programme,Kilifi"
3,hCoV-19/Kenya/C80801/2021,5806,19.416112,2021-02-11,"KEMRI-Wellcome Trust Research Programme,Kilifi"
4,hCoV-19/Kenya/C1751/2020,0,0.000000,2020-06-02,"KEMRI-Wellcome Trust Research Programme,Kilifi"


## Average coverage of sequences

Variant plot

In [21]:
variant_data = pd.read_csv("./data/Kenya_metadata_variants.csv")
variant_data = variant_data.loc[:,["date_collected","pangolin_lineage","VOC"]]
variant_data["date_collected"] = pd.to_datetime(variant_data["date_collected"],format="%d/%m/%Y",infer_datetime_format=True)
variant_data["Year"] = variant_data["date_collected"].dt.to_period("Y")
variant_data["Month"] = variant_data["date_collected"].dt.to_period("M")
variant_data.rename(columns={"VOC":"variant"}, inplace=True)
variant_data = variant_data.groupby(["Month","variant"])[["variant"]].count().rename(columns={"variant":"Frequency"}).reset_index()
variant_data = variant_data[variant_data["variant"] != "Unassigned"]
variant_data["Month"] = variant_data["Month"].astype(str) + "-01"
variant_data["percentage"] = variant_data.groupby(["Month"])[["Frequency"]].apply(lambda x:x/x.sum())#.sum()#.reset_index()
variant_data.tail(n=30)

,Month,variant,Frequency,percentage
55,2021-09-01,Alpha VOC,1,0.003175
56,2021-09-01,Delta VOC,310,0.984127
57,2021-09-01,Non-VOC/VOI,2,0.006349
58,2021-09-01,Omicron VOC,2,0.006349
59,2021-10-01,Alpha VOC,2,0.028986
60,2021-10-01,Delta VOC,64,0.927536
61,2021-10-01,Non-VOC/VOI,3,0.043478
62,2021-11-01,Alpha VOC,2,0.033898
63,2021-11-01,Delta VOC,47,0.796610
64,2021-11-01,Non-VOC/VOI,1,0.016949


In [22]:
variant_data.groupby('Month')[["Frequency"]].sum().reset_index()

,Month,Frequency
0,2020-03-01,29
1,2020-04-01,113
2,2020-05-01,184
3,2020-06-01,105
4,2020-07-01,93
5,2020-08-01,102
6,2020-09-01,55
7,2020-10-01,239
8,2020-11-01,385
9,2020-12-01,173


In [23]:
variant_data.to_csv("../Cemia_Dash/data/variant_data_kenya.tsv",sep = "\t",index=False)

In [168]:
px.bar(variant_data, x = "Month", y = "percentage", color="variant", range_x = ["2020-01-01","2022-12-31"])

In [24]:
#select for Omicron only
import pandas as pd
lineage_data = pd.read_csv("../Cemia_Dash/data/Kenya_metadata_variants.csv")
lineage_data["date_collected"] = pd.to_datetime(lineage_data["date_collected"],format="%d/%m/%Y",infer_datetime_format=True)
omicron = lineage_data.loc[:,["date_collected", "pangolin_lineage", "VOC"]]
omicron = omicron[omicron["VOC"] == "Omicron VOC"]
omicron["Year"] = omicron["date_collected"].dt.to_period("Y")
omicron = omicron[omicron["date_collected"] > "2021-10-31"]
omicron["Month"] = omicron["date_collected"].dt.to_period("M")
omicron.groupby(["Month","pangolin_lineage"])[["pangolin_lineage"]].count().rename(columns={"pangolin_lineage":"Frequency"}).reset_index()
omicron = omicron[omicron["pangolin_lineage"] != "Unassigned"]
#omicron["pangolin_lineage"].unique()

In [25]:
import re
def replace(string):
    if string.startswith("BA.1"):
        new = re.sub("BA.1.*", "BA.1-like", string)
        return new
    elif string.startswith("BA.2"):
        new = re.sub("BA.2.*", "BA.2-like",string)
        return new
    elif string.startswith("BA.3"):
        new = re.sub("BA.3.*", "BA.3-like",string)
        return new
    elif string.startswith("BA.4"):
        new = re.sub("BA.4.*", "BA.4-like",string)
        return new
    elif string.startswith("BA.5"):
        new = re.sub("BA.5.*", "BA.5-like",string)
        return new
    elif string.startswith("BQ.1"):
        new = re.sub("BQ.1.*", "BQ.1-like",string)
        return new
    elif string.startswith("BC"):
        new = re.sub("BC.*", "BC-like",string)
        return new
    elif string.startswith("BE"):
        new = re.sub("BE.*", "BE-like",string)
        return new
    elif string.startswith("BF"):
        new = re.sub("BF.*", "BF-like",string)
        return new
    elif string.startswith("XBB"):
        new = re.sub("XBB.*", "XBB-like",string)
        return new
    else:
        return string
    
omicron["pangolin_lineage"] =  omicron["pangolin_lineage"].apply(lambda x: replace(x))

#B.1.1 was seen way back and needs to be dropped

omicron = omicron[omicron["pangolin_lineage"] != "B.1.1"]
omicron = omicron.groupby(["Month","pangolin_lineage"])[["pangolin_lineage"]].count().rename(columns = {"pangolin_lineage":"Frequency"}).reset_index()
omicron["percentage"] = omicron.groupby(["Month"])[["Frequency"]].apply(lambda x:x/x.sum())#.sum()#.reset_index()
omicron["Month"] = omicron["Month"].astype(str) + "-01"

In [26]:
omicron.to_csv("../Cemia_Dash/data/omicron_data.tsv",sep="\t",index=False)

## Preprocessing CEMia

In [28]:
global_lineage_data = pd.read_table("../data/global_until_20230115.tsv")

data = global_lineage_data[global_lineage_data["region"] == "Europe"]

africa = data.groupby(["pangolin_lineage"])[["pangolin_lineage"]].count().\
        rename(columns = {"pangolin_lineage":"Frequency"}).\
        sort_values("Frequency", ascending=False).reset_index()
        
africa["percent"] = round((africa["Frequency"]/africa["Frequency"].sum()) *100,1)
         ,)



In [51]:


pcolor = "#FFFAFA"
pcolor_white = "white"
margin = dict(l=20, r=25, t=20, b=20)
gridcolor="lightgray"
global_lineage_data = pd.read_table("../data/global_until_20230115.tsv")


def regions_graph(region):
    """Functions to generate top 10 most dominant variants globally by region"""
    data = global_lineage_data[global_lineage_data["region"] == region]
    data = data.groupby(["pangolin_lineage"])[["pangolin_lineage"]].count().\
                rename(columns = {"pangolin_lineage":"Frequency"}).\
                sort_values("Frequency", ascending=False).reset_index()
    total = data["Frequency"].sum()
    data["percent"] = round((data["Frequency"]/data["Frequency"].sum()) *100,1)
    #top_10 = data.iloc[0:10]
    fig = px.bar(data.iloc[0:10], y = "pangolin_lineage",x = "percent")
    fig.update_yaxes(title = None, linecolor = "black",tickfont = dict(size=10),)
    fig.update_traces(width=0.4,marker_color = "#51A2C4",textfont_size=10,textposition="outside")
    fig.update_xaxes(title = "Proportion", linecolor = "black",tickfont = dict(size=10), title_font = dict(size=10),gridcolor = gridcolor, gridwidth = 0.5)
    fig.update_layout(title = region + f" ({total:,} sequences)",title_x=0.3,title_font = dict(size=14),bargap=0.06,yaxis={'categoryorder':'total ascending'}
                        ,plot_bgcolor = pcolor_white,margin=margin)#,margin=dict(l=10, r=100, t=20, b=20)
    return fig


regions_graph("Asia")